In [2]:
%cd /ihome/haizenstein/mez141/ondemand/SwiFUN_clean

/ihome/haizenstein/mez141/ondemand/SwiFUN_clean


/ihome/haizenstein/mez141/.conda/envs/py39/lib/python3.9/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


!ls /root/autodl-tmp/SwiFUN_clean

# Set data directory

In [1]:
import pandas as pd
df = pd.read_csv("/ihome/haizenstein/mez141/ondemand/cleaned_data/metadata/your_task.csv")

old_prefix = "/ihome/haizenstein/mez141/ondemand/SwiFUN_github/ds000030_cleaned_data/"
new_prefix = "/ihome/haizenstein/mez141/ondemand/SwiFUN_clean/ds000030_cleaned_data"

df["label_path"] = df["label_path"].str.replace(old_prefix, new_prefix, regex=False)

df.to_csv("/ihome/haizenstein/mez141/ondemand/cleaned_data/metadata/your_task.csv", index=False)
print("Save as your_task_updated.csv")

Save as your_task_updated.csv


In [4]:
!ln -s /ihome/haizenstein/mez141/ondemand/cleaned_data ./ds000030_cleaned_data

In [5]:
!ls /ihome/haizenstein/mez141/ondemand/SwiFUN_clean/ds000030_cleaned_data/label/sub-10159_label.pt

/ihome/haizenstein/mez141/ondemand/SwiFUN_clean/ds000030_cleaned_data/label/sub-10159_label.pt


# mask file

In [54]:
import torch
import numpy as np
from nilearn import datasets
from scipy.ndimage import zoom

# Download MNI152 2mm standard brain mask
mask_img = datasets.load_mni152_brain_mask(resolution=2)
mask_data = mask_img.get_fdata()  # shape = (91, 109, 91)
print(f"Original mask shape: {mask_data.shape}")

# Resample to (96, 96, 96)
target_shape = (96, 96, 96)
zoom_factors = (
    target_shape[0] / mask_data.shape[0],
    target_shape[1] / mask_data.shape[1],
    target_shape[2] / mask_data.shape[2],
)
print(f"Zoom factors: {zoom_factors}")
resampled_mask = zoom(mask_data, zoom_factors, order=1)  # 线性插值

# Binarize
binary_mask = (resampled_mask > 0.5).astype(np.float32)

# Convert to torch tensor
mask_tensor = torch.tensor(binary_mask)
torch.save(mask_tensor, "./MNI152_mask.pt")
print("Saved MNI152_mask.pt with shape:", mask_tensor.shape)


Original mask shape: (99, 117, 95)
Zoom factors: (0.9696969696969697, 0.8205128205128205, 1.0105263157894737)
Saved MNI152_mask.pt with shape: torch.Size([96, 96, 96])


# Run the model

In [6]:
#test
!python test/module_test_swin4d.py

img_size:  (96, 96, 96, 20)
patch_size:  (6, 6, 6, 1)
patch_dim:  (16, 16, 16, 20)
done


In [7]:
import torch
print("CUDA available:", torch.cuda.is_available())
print("CUDA device count:", torch.cuda.device_count())
if torch.cuda.is_available():
    print("CUDA device name:", torch.cuda.get_device_name(0))

CUDA available: True
CUDA device count: 1
CUDA device name: NVIDIA A100-PCIE-40GB


In [8]:
!nvidia-smi


Thu Aug 21 07:51:52 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 575.57.08              Driver Version: 575.57.08      CUDA Version: 12.9     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-PCIE-40GB          On  |   00000000:C7:00.0 Off |                    0 |
| N/A   29C    P0             33W /  250W |       4MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [ ]:
TRAINER_ARGS="--accelerator gpu --max_epochs 20 --precision 16 --num_nodes 1" #--strategy ddp_find_unused_parameters_false" # devices should be 4 for sbatch
MAIN_ARGS='--loggername tensorboard --id ds000030_test_run1 --dataset_name DS000030 --image_path ./ds000030_cleaned_data --task_path ./ds000030_cleaned_data/label --mask_input  --mask_filename MNI152_mask.pt'#--mask_filename harvard_oxford-wholebrain-maxprob-thr25-2mm.nii.gz' # /global/cfs/cdirs/m4244/junbeom/20249_pickled_map
DATA_ARGS='--batch_size 2 --eval_batch_size 16 --num_workers 8'
DEFAULT_ARGS='--project_name ds000030_experiment '
OPTIONAL_ARGS='--cope 1 --c_multiplier 2 --clf_head_version v1 --downstream_task tfMRI_3D --use_scheduler --gamma 0.5 --cycle 0.7 --loss_type mse --last_layer_full_MSA True '  
RESUME_ARGS="" 

!python project/main.py $TRAINER_ARGS \
  $MAIN_ARGS \
  $DEFAULT_ARGS \
  $DATA_ARGS \
  $OPTIONAL_ARGS \
  $RESUME_ARGS \
  --dataset_split_num 1 \
  --seed 1 \
  --learning_rate 5e-5 \
  --model swin4d_ver7 \
  --attn_drop_rate 0.3 \
  --depth 2 2 2 \
  --embed_dim 24 \
  --sequence_length 20 \
  --swift_window_size 4 4 4 6 \
  --swift_patch_size 6 6 6 1 \
  --img_size 96 96 96 20 \
  --input_scaling_method none 

Found 98 subjects in img directory
Metadata contains 51 entries
Successfully loaded 51 valid subject-label pairs
[DEBUG in make_subject_dict] final_dict has 51 entries

=== DS000030._set_data RESULT ===
Total samples generated: 244
Unique subjects: 35
Dataset initialized with 244 samples from 35 subjects

=== DS000030._set_data RESULT ===
Total samples generated: 49
Unique subjects: 7
Dataset initialized with 49 samples from 7 subjects

=== DS000030._set_data RESULT ===
Total samples generated: 63
Unique subjects: 9
Dataset initialized with 63 samples from 9 subjects
number of train_subj: 35
number of val_subj: 7
number of test_subj: 9
length of train_idx: 244
length of val_idx: 49
length of test_idx: 63
[rank: 0] Global seed set to 1
not using gradient clipping
Namespace(seed=1, dataset_name='DS000030', downstream_task='tfMRI_3D', downstream_task_type='default', loggername='tensorboard', project_name='ds000030_experiment', resume_ckpt_path=None, load_model_path=None, test_only=False, 